# Notebook

This notebook provides interactive visualization of flow data using PyVista. It allows you to visualize VTK files showing velocity fields over time with optional biomodel integration.

## Setup and Imports

Import all necessary libraries for data visualization.

In [ ]:
import os
from pathlib import Path

import pyvista as pv
import numpy as np
import matplotlib.pyplot as plt

# For notebook integration
pv.set_jupyter_backend("pythreejs")  # Use pythreejs backend for Jupyter
%matplotlib inline

## Utility Functions

Defining helper functions for visualization from the original script.

In [ ]:
def load_biomodel(biomodel_path):
    """Load the biomodel from a VTK file."""
    if not os.path.exists(biomodel_path):
        print(f"Warning: Biomodel file {biomodel_path} not found.")
        return None

    return pv.read(biomodel_path)


def extract_aorta(dataset, biomodel_data):
    """Extract the aorta region from the dataset using the biomodel."""
    # This is a simplified version - in a real case, you might need more complex extraction logic
    if biomodel_data is not None:
        # Here you would typically use the biomodel to extract the relevant region
        # For simplicity, we'll just return the dataset as-is
        print("Using biomodel for region extraction (simplified)")
    return dataset


def calculate_velocity_statistics(dataset):
    """Calculate statistics of velocity magnitude in the dataset."""
    # Check if 'Velocity' array exists
    if "Velocity" in dataset.array_names:
        velocity = dataset["Velocity"]
        # Calculate velocity magnitude if needed
        if "VelocityMagnitude" not in dataset.array_names:
            magnitudes = np.linalg.norm(velocity, axis=1)
            dataset["VelocityMagnitude"] = magnitudes
        else:
            magnitudes = dataset["VelocityMagnitude"]

        return {
            "mean": np.mean(magnitudes),
            "peak": np.max(magnitudes),
            "min": np.min(magnitudes),
            "median": np.median(magnitudes),
            "std": np.std(magnitudes),
        }
    else:
        print("Warning: No 'Velocity' array found in dataset")
        return {"mean": 0, "peak": 0, "min": 0, "median": 0, "std": 0}


def render_volume(dataset):
    """Create volume representation for rendering."""
    # Make sure VelocityMagnitude exists
    if (
        "VelocityMagnitude" not in dataset.array_names
        and "Velocity" in dataset.array_names
    ):
        velocity = dataset["Velocity"]
        magnitudes = np.linalg.norm(velocity, axis=1)
        dataset["VelocityMagnitude"] = magnitudes

    # Clone dataset to avoid modifying original
    volume = dataset.copy()
    return volume


def generate_streamlines(dataset, center=[130, 260, 40], radius=20):
    """Generate streamlines starting from points around a center position."""
    # Create seed points (a sphere of points around the center)
    sphere = pv.Sphere(
        radius=radius, center=center, phi_resolution=10, theta_resolution=10
    )

    # Generate streamlines
    streamlines = dataset.streamlines(
        vectors="Velocity",  # Use velocity vectors for streamlines
        source=sphere,  # Start from these seed points
        max_time=200,  # Maximum integration time
        integration_direction="both",  # Integrate in both directions
        initial_step_length=1.0,  # Initial step size for integration
        max_steps=1000,  # Maximum number of steps
        terminal_speed=0.01,  # Stop integration when speed falls below this value
    )

    return streamlines, sphere

## Visualization Class

Creating a class to manage the visualization callbacks and UI elements.

In [ ]:
class VisualizationCallbacks:
    """Container class for visualization callbacks to manage interactive UI elements."""

    def __init__(self, plotter):
        self.plotter = plotter
        self.actors = {}  # Dictionary to store actor references for UI control

    def add_actor(self, name, actor):
        """Store an actor reference for later manipulation and visibility control."""
        self.actors[name] = actor

    def toggle_volume(self, flag):
        """Toggle visibility of the volume rendering based on checkbox state."""
        if "volume" in self.actors:
            self.actors["volume"].SetVisibility(flag)
            self.plotter.update()

    def toggle_streamlines(self, flag):
        """Toggle visibility of the streamlines based on checkbox state."""
        if "streamlines" in self.actors:
            self.actors["streamlines"].SetVisibility(flag)
            self.plotter.update()

    def sphere_callback(self, pos):
        """Print the current position of the sphere widget (used for debugging)."""
        print(f"Sphere position: {pos}")

## Data Loading and Preparation

Load the data files and prepare them for visualization.

In [ ]:
# Parameters (you can change these as needed)
data_dir = Path("../data")
timestep = 85  # Default timestep
biomodel_path = Path("../assets/biomodel.vtk")

# List available timesteps
available_files = [f for f in os.listdir(data_dir) if f.startswith("data.vts.")]
available_timesteps = [int(f.split(".")[-1]) for f in available_files]
available_timesteps.sort()

print(f"Available timesteps: {available_timesteps}")

# Construct the full filepath for the selected timestep data
filename = f"data.vts.{timestep}"
filepath = data_dir / filename

# Validate that the requested timestep file exists
if not filepath.exists():
    print(f"Error: File {filepath} not found.")
    # List available timesteps to help the user
    print(f"Available timesteps: {available_timesteps}")
else:
    print(f"Using timestep {timestep} from file: {filepath}")

## Load and Process Data

Load the VTS and biomodel data, and extract relevant information.

In [ ]:
# Load data files
try:
    # Load the VTS dataset for the selected timestep
    dataset = pv.read(str(filepath), force_ext=".vts")
    print(f"Successfully loaded dataset from {filepath}")

    # Load anatomical model
    biomodel_data = load_biomodel(biomodel_path)
    if biomodel_data is not None:
        print(f"Successfully loaded biomodel from {biomodel_path}")

    # Extract aorta region from dataset
    aorta = extract_aorta(dataset, biomodel_data)

    # Display available data arrays for debugging and information
    print("\nData arrays in aorta dataset:")
    for name in aorta.array_names:
        print(f"  - {name}")

    # Calculate and display velocity statistics for the current timestep
    velocity_stats = calculate_velocity_statistics(aorta)
    print("\nVelocity Statistics:")
    print(f"  Mean Velocity: {velocity_stats['mean']:.2f} cm/s")
    print(f"  Peak Velocity: {velocity_stats['peak']:.2f} cm/s")
    print(f"  Min Velocity: {velocity_stats['min']:.2f} cm/s")
    print(f"  Median Velocity: {velocity_stats['median']:.2f} cm/s")
    print(f"  Std Deviation: {velocity_stats['std']:.2f} cm/s")

except Exception as e:
    print(f"Error loading data: {e}")

## Generate Visualization Elements

Prepare the volume rendering and streamlines for visualization.

In [ ]:
# Define center point for streamline generation (coordinates in the dataset space)
center = [130, 260, 40]

# Generate volume and streamline representations
volume = render_volume(aorta)  # Create volume representation of the aorta
streamlines, source = generate_streamlines(
    aorta, center
)  # Generate flow streamlines and seed points

print(f"Generated volume rendering from aorta data")
print(
    f"Generated streamlines with {streamlines.n_points} points and {streamlines.n_cells} cells"
)

## Interactive Visualization

Create an interactive visualization with PyVista plotter.

In [ ]:
# Initialize the PyVista plotter for 3D visualization
plotter = pv.Plotter(notebook=True)

# Create callback manager for interactive UI elements
callbacks = VisualizationCallbacks(plotter)

# Add velocity statistics text overlay to the visualization
plotter.add_text(
    f"Mean Velocity: {velocity_stats['mean']:.2f} cm/s\nPeak Velocity: {velocity_stats['peak']:.2f} cm/s",
    position="upper_left",
    font_size=12,
    shadow=True,
)

# Add biomodel as a wireframe to provide anatomical context
if biomodel_data is not None:
    plotter.add_mesh(
        biomodel_data,
        color="white",
        opacity=0.2,
        label="Biomodel",
        show_edges=True,
        edge_color="black",
        style="wireframe",
    )

# Add volume rendering of velocity magnitude
volume_plot = plotter.add_volume(
    volume,
    scalars="VelocityMagnitude",  # Color by velocity magnitude
    clim=[0, 150],  # Color range in cm/s
    opacity="linear",  # Linear opacity mapping
    scalar_bar_args=dict(
        title="Velocity [cm/s]",
        n_labels=6,
        vertical=True,
    ),
    mapper="smart",  # Smart volume mapper for better rendering
    cmap="rainbow",  # Color map for velocity visualization
    blending="composite",  # Composite blending mode for volume rendering
)
callbacks.add_actor("volume", volume_plot)

# Add streamlines visualization colored by velocity
streamlines_actor = plotter.add_mesh(
    streamlines,
    line_width=5.0,  # Increased line width for better visibility in notebook
    label="Streamlines",
    cmap="rainbow",  # Color map matching the volume rendering
    show_scalar_bar=False,  # Don't show duplicate scalar bar
)
callbacks.add_actor("streamlines", streamlines_actor)

# Add visualization of streamline seed points
source_actor = plotter.add_mesh(
    source,
    color="yellow",
    point_size=8,
    render_points_as_spheres=True,
    label="Seed Points",
)
callbacks.add_actor("seeds", source_actor)

# Configure camera and grid for optimal viewing
plotter.view_isometric()  # Set isometric view
plotter.show_grid(font_size=12)
plotter.add_axes()

# Set window properties and display the visualization
plotter.background_color = "black"
plotter.window_size = [800, 600]  # Set window dimensions

# Render the scene
plotter.show()

## Visualization Controls

In the traditional script, these would be checkbox buttons for toggling visualization elements. In a notebook environment, we'll use interactive widgets instead.

In [ ]:
# For a more interactive experience, we can use ipywidgets
try:
    import ipywidgets as widgets
    from IPython.display import display

    # Create a new plotter for widget-controlled visualization
    p = pv.Plotter(notebook=True)
    p.add_mesh(
        volume,
        scalars="VelocityMagnitude",
        clim=[0, 150],
        opacity="linear",
        cmap="rainbow",
    )

    # Create toggle switches
    volume_toggle = widgets.Checkbox(value=True, description="Show Volume")
    streamlines_toggle = widgets.Checkbox(value=True, description="Show Streamlines")
    seed_toggle = widgets.Checkbox(value=True, description="Show Seeds")

    # Define callback functions
    def update_plot(change):
        p.clear()
        if volume_toggle.value:
            p.add_mesh(
                volume,
                scalars="VelocityMagnitude",
                clim=[0, 150],
                opacity="linear",
                cmap="rainbow",
            )
        if streamlines_toggle.value:
            p.add_mesh(streamlines, line_width=3.0, cmap="rainbow")
        if seed_toggle.value:
            p.add_mesh(
                source, color="yellow", point_size=8, render_points_as_spheres=True
            )
        p.update()

    # Connect callbacks
    volume_toggle.observe(update_plot, names="value")
    streamlines_toggle.observe(update_plot, names="value")
    seed_toggle.observe(update_plot, names="value")

    # Display widgets
    display(widgets.HBox([volume_toggle, streamlines_toggle, seed_toggle]))
    p.show()

except ImportError:
    print("ipywidgets not available. Interactive controls disabled.")

## Time Series Visualization

Function to visualize different timesteps of the flow data.

In [ ]:
def visualize_timestep(timestep):
    """Visualize a specific timestep from the dataset."""
    # Construct the filepath
    filename = f"data.vts.{timestep}"
    filepath = data_dir / filename

    if not filepath.exists():
        print(f"Error: File {filepath} not found.")
        return None

    # Load and process data
    dataset = pv.read(str(filepath), force_ext=".vts")
    aorta = extract_aorta(dataset, biomodel_data)
    volume = render_volume(aorta)
    streamlines, _ = generate_streamlines(aorta, center)

    # Calculate velocity statistics
    velocity_stats = calculate_velocity_statistics(aorta)

    # Create plotter
    p = pv.Plotter(notebook=True)
    p.add_text(
        f"Timestep: {timestep}\nMean Velocity: {velocity_stats['mean']:.2f} cm/s\nPeak: {velocity_stats['peak']:.2f} cm/s",
        position="upper_left",
    )

    # Add meshes
    p.add_mesh(
        volume,
        scalars="VelocityMagnitude",
        clim=[0, 150],
        opacity="linear",
        cmap="rainbow",
    )
    p.add_mesh(streamlines, line_width=3.0, cmap="rainbow", show_scalar_bar=False)

    p.view_isometric()
    p.show_grid()

    return p, velocity_stats

## Interactive Timestep Selection

Use a slider to select different timesteps (if ipywidgets is available).

In [ ]:
try:
    import ipywidgets as widgets
    from IPython.display import display, clear_output

    # Create slider for timestep selection
    if len(available_timesteps) > 0:
        timestep_slider = widgets.IntSlider(
            min=min(available_timesteps),
            max=max(available_timesteps),
            step=available_timesteps[1] - available_timesteps[0]
            if len(available_timesteps) > 1
            else 1,
            value=available_timesteps[0],
            description="Timestep:",
            continuous_update=False,
        )

        output = widgets.Output()

        # Function to update visualization when slider changes
        def on_timestep_change(change):
            with output:
                clear_output(wait=True)
                print(f"Loading timestep {change['new']}...")
                p, stats = visualize_timestep(change["new"])
                if p is not None:
                    p.show()

        # Register callback
        timestep_slider.observe(on_timestep_change, names="value")

        # Initial display
        display(timestep_slider)
        display(output)

        # Trigger initial visualization
        on_timestep_change({"new": timestep_slider.value})
    else:
        print("No timesteps available for visualization.")

except ImportError:
    print("ipywidgets not available. Interactive timestep selection disabled.")

## Velocity Statistics Analysis

Analyze velocity statistics across multiple timesteps.

In [ ]:
# Sample a subset of timesteps for analysis
sample_timesteps = available_timesteps[
    ::3
]  # Take every 3rd timestep to reduce computation
if len(sample_timesteps) > 10:
    sample_timesteps = sample_timesteps[:10]  # Limit to 10 samples max

print(f"Analyzing velocity statistics for timesteps: {sample_timesteps}")

# Collect statistics
timesteps = []
mean_velocities = []
peak_velocities = []

for ts in sample_timesteps:
    try:
        filepath = data_dir / f"data.vts.{ts}"
        dataset = pv.read(str(filepath), force_ext=".vts")
        aorta = extract_aorta(dataset, biomodel_data)
        stats = calculate_velocity_statistics(aorta)

        timesteps.append(ts)
        mean_velocities.append(stats["mean"])
        peak_velocities.append(stats["peak"])

        print(
            f"Timestep {ts}: Mean = {stats['mean']:.2f} cm/s, Peak = {stats['peak']:.2f} cm/s"
        )
    except Exception as e:
        print(f"Error processing timestep {ts}: {e}")

# Plot velocity statistics over time
plt.figure(figsize=(12, 6))
plt.plot(timesteps, mean_velocities, "o-", label="Mean Velocity")
plt.plot(timesteps, peak_velocities, "s-", label="Peak Velocity")
plt.xlabel("Timestep")
plt.ylabel("Velocity (cm/s)")
plt.title("Velocity Statistics Over Time")
plt.grid(True)
plt.legend()
plt.show()

## Conclusion

This notebook provides interactive visualization of flow data using PyVista. You can:

1. View volume renderings of velocity fields
2. Generate and visualize streamlines showing flow directions
3. Toggle different visualization elements on and off
4. Explore different timesteps to see how the flow evolves over time
5. Analyze velocity statistics across the dataset

The interactive nature of Jupyter makes it easier to explore different aspects of the data compared to the original script.